What is the average priced paid per amenity
What is the average price paid per top 5 combined amenities
Can we predict what the top amenities are or what properties are being sold based on amenities
Why is TV missing from top of list.  -- Manually checked if most properties have TV

In [2]:
import pandas as pd
import nltk
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from ast import literal_eval

import numpy as np
# nltk.download('punkt')

# pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_row',None)

In [3]:
Boston = pd.read_csv('../Data_Files/BostonGZ_listings.csv')
df2 = pd.read_csv('../Data_Files/Boston_listings.csv')
Boston['amenities'].dtype

dtype('O')

In [57]:
##Due to file type being csv, the column is converted to a string automatically.
##literal_eval is used to convert string to list format.

Boston_amenities_list = []
for items in Boston.amenities:
    Boston_amenities_list.append(literal_eval(items))

Boston_amenities_list[34]
##Count to see if TV in list

##TV should be the estimated number one amenity
##Code to combine TV and put in list.
updated_boston_amenities = []
Boston_amenities_list = [['TV' if 'TV' in item else item for item in lst] for lst in Boston_amenities_list ]

#need to account for duplicate TV in lists or other duplicates
#len(Boston_amenities_list)
Bost_amen_list=[]
for l in Boston_amenities_list:
    Bost_amen_list.append([*set(l)])
Bost_amen_list[0]


['Oven',
 'Hangers',
 'Shampoo',
 'Bed linens',
 'Hair dryer',
 'Refrigerator',
 'Hot water',
 'Dishwasher',
 'Cooking basics',
 'Extra pillows and blankets',
 'Stove',
 'Dedicated workspace',
 'Coffee maker',
 'Microwave',
 'Iron',
 'Kitchen',
 'Air conditioning',
 'Long term stays allowed',
 'Smoke alarm',
 'Free parking on premises',
 'Essentials',
 'Free street parking',
 'Heating',
 'Patio or balcony',
 'Carbon monoxide alarm',
 'Wifi',
 'Washer',
 'TV',
 'Dishes and silverware',
 'Dryer']

In [58]:
##20 Most Common Amenities
##Why is TV missing
## Assumption was made that TV was going to be number one.
##After some data wrangling TV is estimated at number 9

result_list = []
amenities_counter = Counter()

for amens in Bost_amen_list:
    amenities_counter.update(amens)
amenities_counter.most_common(100)

[('Wifi', 3364),
 ('Smoke alarm', 3337),
 ('Long term stays allowed', 3276),
 ('Carbon monoxide alarm', 3126),
 ('Kitchen', 3104),
 ('Essentials', 3099),
 ('Hangers', 3031),
 ('Hair dryer', 2913),
 ('TV', 2898),
 ('Iron', 2880),
 ('Heating', 2851),
 ('Shampoo', 2693),
 ('Air conditioning', 2677),
 ('Hot water', 2592),
 ('Microwave', 2397),
 ('Refrigerator', 2386),
 ('Coffee maker', 2317),
 ('Dedicated workspace', 2275),
 ('Dishes and silverware', 2198),
 ('Cooking basics', 2176),
 ('Washer', 2161),
 ('Bed linens', 2152),
 ('Dryer', 2056),
 ('Fire extinguisher', 1980),
 ('Oven', 1835),
 ('Stove', 1736),
 ('Dishwasher', 1497),
 ('Extra pillows and blankets', 1345),
 ('First aid kit', 1242),
 ('Private entrance', 1185),
 ('Free street parking', 1005),
 ('Luggage dropoff allowed', 891),
 ('Security cameras on property', 806),
 ('Lockbox', 800),
 ('Shower gel', 763),
 ('Bathtub', 737),
 ('Lock on bedroom door', 712),
 ('Keypad', 710),
 ('Free parking on premises', 645),
 ('Elevator', 638),


In [59]:
#top combined amenities
n = 4
n_gram_counter = Counter()
for gram in Bost_amen_list:
    gram_list = nltk.ngrams(gram,n)
    n_gram_counter.update(gram_list)
n_gram_counter.most_common(5)

[(('Kitchen', 'Air conditioning', 'Long term stays allowed', 'Smoke alarm'),
  1055),
 (('Iron', 'Kitchen', 'Air conditioning', 'Long term stays allowed'), 820),
 (('Oven', 'Hangers', 'Shampoo', 'Bed linens'), 790),
 (('Heating', 'Carbon monoxide alarm', 'Wifi', 'Washer'), 783),
 (('Private entrance', 'Hair dryer', 'Refrigerator', 'Hot water'), 742)]

In [60]:
# amenities_counter.most_common(100)
top_100_amenities=[word for word,cnt in amenities_counter.most_common(100)]
df = pd.DataFrame(columns=top_100_amenities)

zeros = [0]* 100

counter = 0

for element in Bost_amen_list:
    df.loc[-1]= zeros
    df = df.reset_index(drop=True)
    for col in df.columns:
        for item in element:
            if item == col:
                df.at[counter,col] =1
    counter+=1
    
df            

,Wifi,Smoke alarm,Long term stays allowed,Carbon monoxide alarm,Kitchen,Essentials,Hangers,Hair dryer,TV,Iron,Heating,Shampoo,Air conditioning,Hot water,Microwave,Refrigerator,Coffee maker,Dedicated workspace,Dishes and silverware,Cooking basics,Washer,Bed linens,Dryer,Fire extinguisher,Oven,Stove,Dishwasher,Extra pillows and blankets,First aid kit,Private entrance,Free street parking,Luggage dropoff allowed,Security cameras on property,Lockbox,Shower gel,Bathtub,Lock on bedroom door,Keypad,Free parking on premises,Elevator,Baking sheet,Gym,Freezer,Cleaning products,Toaster,Body soap,Wine glasses,Patio or balcony,Dining table,Hot water kettle,Paid parking off premises,BBQ grill,Conditioner,Backyard,Room-darkening shades,Pack ’n play/Travel crib,Pool,Smart lock,Ethernet connection,Indoor fireplace,Laundromat nearby,Paid parking on premises,Outdoor furniture,Portable fans,Central heating,Breakfast,Host greets you,Clothing storage: closet,Outdoor dining area,Private patio or balcony,Keurig coffee machine,Crib,Ceiling fan,Single level home,Window AC unit,Mini fridge,High chair,Shared patio or balcony,Children’s books and toys,Central air conditioning,Stainless steel oven,Paid washer – In building,Clothing storage,Paid dryer – In building,Barbecue utensils,Cleaning before checkout,Free dryer – In unit,Window guards,Paid parking garage off premises,Free washer – In unit,Building staff,Board games,Pocket wifi,Drying rack for clothing,Children’s dinnerware,EV charger,Radiant heating,Dryer – In unit,Washer – In unit,Safe
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,0,0,1,1,1,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3457,1,1,1,1,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3458,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3459,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,1,1,1,1,1,1,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3460,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [61]:
# ##Set of all unique amenities.
# amenities = list(Boston.amenities)
# amenities_string = " ".join(amenities)
# amenities_string = amenities_string.replace('{', '')
# amenities_string = amenities_string.replace('}', ',')
# amenities_string = amenities_string.replace('[', '')
# amenities_string = amenities_string.replace(']', ',')
# amenities_string = amenities_string.replace('"', '')
# amenities_set = [x.strip() for x in amenities_string.split(',')]
# # amenities_set = set(amenities_set)
# # amenities_set

# amenities_set

In [62]:
Boston.head()
Boston_id_price = Boston[["host_id","price"]]
Boston_id_price.head()

,host_id,price
0,4804,$132.00
1,8229,$149.00
2,8229,$250.00
3,26988,$110.00
4,26988,$100.00


In [63]:
Boston_concat_price_amenites = pd.concat([Boston_id_price, df], axis=1)
Boston_concat_price_amenites.head()

,host_id,price,Wifi,Smoke alarm,Long term stays allowed,Carbon monoxide alarm,Kitchen,Essentials,Hangers,Hair dryer,TV,Iron,Heating,Shampoo,Air conditioning,Hot water,Microwave,Refrigerator,Coffee maker,Dedicated workspace,Dishes and silverware,Cooking basics,Washer,Bed linens,Dryer,Fire extinguisher,Oven,Stove,Dishwasher,Extra pillows and blankets,First aid kit,Private entrance,Free street parking,Luggage dropoff allowed,Security cameras on property,Lockbox,Shower gel,Bathtub,Lock on bedroom door,Keypad,Free parking on premises,Elevator,Baking sheet,Gym,Freezer,Cleaning products,Toaster,Body soap,Wine glasses,Patio or balcony,Dining table,Hot water kettle,Paid parking off premises,BBQ grill,Conditioner,Backyard,Room-darkening shades,Pack ’n play/Travel crib,Pool,Smart lock,Ethernet connection,Indoor fireplace,Laundromat nearby,Paid parking on premises,Outdoor furniture,Portable fans,Central heating,Breakfast,Host greets you,Clothing storage: closet,Outdoor dining area,Private patio or balcony,Keurig coffee machine,Crib,Ceiling fan,Single level home,Window AC unit,Mini fridge,High chair,Shared patio or balcony,Children’s books and toys,Central air conditioning,Stainless steel oven,Paid washer – In building,Clothing storage,Paid dryer – In building,Barbecue utensils,Cleaning before checkout,Free dryer – In unit,Window guards,Paid parking garage off premises,Free washer – In unit,Building staff,Board games,Pocket wifi,Drying rack for clothing,Children’s dinnerware,EV charger,Radiant heating,Dryer – In unit,Washer – In unit,Safe
0,4804,$132.00,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,8229,$149.00,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,0,0,1,1,1,1,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,8229,$250.00,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,26988,$110.00,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,26988,$100.00,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [64]:
#Convert dollar amounts to floats
Boston_concat_price_amenites['price']  = Boston_concat_price_amenites['price'].replace('[\$,]', '', regex=True).astype(float)

#test average or median
median = round(Boston_concat_price_amenites.loc[Boston_concat_price_amenites['Wifi'] == 1, 'price'].median(),2)
mean = round(Boston_concat_price_amenites.loc[Boston_concat_price_amenites['Wifi'] == 1, 'price'].mean(),2)

print("Median: ",median, "  Mean: ",mean)

Median:  150.0   Mean:  197.32


In [65]:
#This cell cannot be ran more than once or it inserts an extra row.

average_price=[]
for col in df.columns:
    average_price.append(round(Boston_concat_price_amenites.loc[Boston_concat_price_amenites[col] == 1, 'price'].mean(),2))

average_price.insert(0,float("nan"))
average_price.insert(0,float("nan"))

average_price

Boston_concat_price_amenites.loc[-1] = average_price  # adding a row
Boston_concat_price_amenites.index = Boston_concat_price_amenites.index + 1  # shifting index
Boston_concat_price_amenites = Boston_concat_price_amenites.sort_index() 

Boston_concat_price_amenites.head()

,host_id,price,Wifi,Smoke alarm,Long term stays allowed,Carbon monoxide alarm,Kitchen,Essentials,Hangers,Hair dryer,TV,Iron,Heating,Shampoo,Air conditioning,Hot water,Microwave,Refrigerator,Coffee maker,Dedicated workspace,Dishes and silverware,Cooking basics,Washer,Bed linens,Dryer,Fire extinguisher,Oven,Stove,Dishwasher,Extra pillows and blankets,First aid kit,Private entrance,Free street parking,Luggage dropoff allowed,Security cameras on property,Lockbox,Shower gel,Bathtub,Lock on bedroom door,Keypad,Free parking on premises,Elevator,Baking sheet,Gym,Freezer,Cleaning products,Toaster,Body soap,Wine glasses,Patio or balcony,Dining table,Hot water kettle,Paid parking off premises,BBQ grill,Conditioner,Backyard,Room-darkening shades,Pack ’n play/Travel crib,Pool,Smart lock,Ethernet connection,Indoor fireplace,Laundromat nearby,Paid parking on premises,Outdoor furniture,Portable fans,Central heating,Breakfast,Host greets you,Clothing storage: closet,Outdoor dining area,Private patio or balcony,Keurig coffee machine,Crib,Ceiling fan,Single level home,Window AC unit,Mini fridge,High chair,Shared patio or balcony,Children’s books and toys,Central air conditioning,Stainless steel oven,Paid washer – In building,Clothing storage,Paid dryer – In building,Barbecue utensils,Cleaning before checkout,Free dryer – In unit,Window guards,Paid parking garage off premises,Free washer – In unit,Building staff,Board games,Pocket wifi,Drying rack for clothing,Children’s dinnerware,EV charger,Radiant heating,Dryer – In unit,Washer – In unit,Safe
0,NaN,NaN,197.32,194.27,198.43,193.35,202.37,204.71,196.32,205.11,214.38,203.26,195.2,209.74,203.23,200.6,200.26,199.24,201.81,187.88,195.14,210.87,201.8,211.99,202.0,208.67,202.48,203.87,218.93,213.73,203.93,227.86,152.87,207.37,214.1,203.56,238.67,233.71,105.59,170.49,180.5,272.93,230.18,285.33,217.7,202.16,213.35,228.29,213.02,198.71,204.58,201.38,249.61,242.59,229.81,168.53,246.33,279.19,269.92,255.54,249.04,232.73,198.79,276.1,233.73,174.81,212.42,179.06,156.62,191.45,224.2,267.11,201.16,336.66,205.74,231.55,159.64,149.67,277.68,127.47,307.63,240.0,212.11,143.28,260.58,140.75,270.84,137.13,254.67,277.0,242.82,236.95,280.51,229.17,240.62,193.08,285.31,259.81,211.63,308.03,293.66,255.73
1,4804.0,132.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.0,1.00,1.00,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.0,1.00,1.0,0.00,1.00,1.00,1.00,1.00,0.00,0.00,1.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,1.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,8229.0,149.0,1.00,1.00,1.00,1.00,0.00,1.00,1.00,1.00,1.00,1.00,1.0,1.00,1.00,1.0,1.00,1.00,1.00,1.00,1.00,0.00,1.0,1.00,1.0,1.00,0.00,0.00,0.00,1.00,1.00,1.00,1.00,1.00,0.0,0.00,0.00,1.00,0.00,1.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,8229.0,250.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.0,1.00,1.00,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.0,0.00,1.0,1.00,1.00,1.00,1.00,0.00,0.00,0.00,1.00,1.00,0.0,0.00,0.00,0.00,0.00,1.00,0.0,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,26988.0,110.0,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.0,0.00,1.00,1.0,1.00,1.00,1.00,1.00,1.00,1.00,1.0,0.00,1.0,1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,1.00,0.00,0.00,0.0,0.00,0.00,0.0